# 6. Serving
 --------------------------------------------------------------------

Deploy the model created in the previous notebook and predict the output of the data coming from `serving-stream`. The model execution and results are tracked using MLRun.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

In [ ]:
%run config.py

## Load project

In [1]:
from os import path, getenv
from mlrun import load_project, mount_v3io
import nuclio

In [2]:
project_name = '-'.join(filter(None, [rapid, getenv('V3IO_USERNAME', None)]))
project_path = path.abspath('conf')
project = load_project(name=project_name, context=project_path, init_git=False)

print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/rapid-proto/conf
Project name: rapid-prototype-michaelk


### Add Serving Functions from the MLRun Functions Marketplace <a id="gs-ml-pipeline-add-functions"></a>


In [3]:
project.set_function('hub://model_server:development', 'serving')

input_stream = f'http://v3io-webapi:8081/users/{getenv("V3IO_USERNAME")}/examples/rapid-prototype/serving-stream@modelserver'
partitions = list(range(0,8))

serving = project.func('serving').apply(mount_v3io())
serving.add_model('rapid_proto', '/User/rapid-proto/artifacts/model/3/model.pkl')
serving.set_envs({'INFERENCE_STREAM' : path.join('users', getenv('V3IO_USERNAME'), 'examples/rapid-prototype/inference-stream')})

serving.add_trigger('serving_stream', nuclio.triggers.V3IOStreamTrigger(url=input_stream, partitions=partitions))
serving.spec.config.pop('spec.triggers.http')

serving.spec.min_replicas = 1

In [4]:
serving_addr = serving.deploy()

> 2020-08-03 16:33:54,338 [info] deploy started
[nuclio] 2020-08-03 16:35:09,589 (info) Build complete
[nuclio] 2020-08-03 16:35:28,852 done creating rapid-prototype-mk-sklearn-server, function address: 192.168.226.12:32259


'http://192.168.226.12:32259'

### Trigger the seving func

In [5]:
import requests
import json

feat = [7,799,47,17,143,1560,7,164,11,810.0,162.0,5.0,782.0]
json = json.dumps({'instances': np.array(feat).reshape(1,-1).tolist()})

resp = requests.post(url=f'{serving_addr}/rapid_proto/predict', 
                     json = json)
print(resp.status_code)
print(resp.text)

200
[0]
